In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import random

In [2]:
# Paths
ROOT = Path('.')
DATA_DIR = ROOT / 'data'
OUTPUT_DIR = ROOT / 'output'
DATA_DIR.mkdir(exist_ok=True)
OUTPUT_DIR.mkdir(exist_ok=True)

# Filenames
students_file = DATA_DIR / 'students.xlsx'
choices_file = DATA_DIR / 'choices.xlsx'
programmes_file = DATA_DIR / 'programmes.xlsx'
result_file = OUTPUT_DIR / 'admission_results.xlsx'

In [3]:
# Generate >= 50 fictitious students with UG2025 IDs
num_students = 60
first_names = ['Ama','Kofi','Abena','Yaw','Akosua','Kwame','Esi','Kojo','Efua','Kweku','Adwoa','Kwaku','Yaa','Nana','Sika']
last_names = ['Mensah','Boateng','Addo','Owusu','Annan','Osei','Doku','Amankwah','Baah','Asare','Dankyi','Gyasi']

def make_id(n):
    return f'UG2025{n:04d}'

records = []
for i in range(1, num_students+1):
    sid = make_id(i)
    name = random.choice(first_names) + ' ' + random.choice(last_names)
    # Generate aggregate between 3 (best) and 30 (worst)
    # bias towards middle values a bit
    agg = random.choices(range(3,31), weights=[abs(17 - x) for x in range(3,31)], k=1)[0]
    records.append({'StudentID': sid, 'Name': name, 'Aggregate': agg})

students_df = pd.DataFrame(records)
# Save to Excel (sheet1)
students_df.to_excel(students_file, index=False, sheet_name='students')
students_df.head()

,StudentID,Name,Aggregate
0,UG20250001,Sika Boateng,26
1,UG20250002,Yaa Gyasi,10
2,UG20250003,Esi Mensah,8
3,UG20250004,Esi Annan,3
4,UG20250005,Kofi Dankyi,8


In [4]:
# Generate choices for each student (first, second, third) from a list of sample schools/programmes
program_list = [
    'UG-CS', 'UG-EE', 'UG-BUS', 'UG-MED', 'UG-ENG', 'UG-ARCH', 'UG-LAW', 'UG-ARTS', 'UG-SOC', 'UG-BIO', 'UG-CHEM', 'UG-MATH'
 ]
choices = []
for sid in students_df['StudentID']:
    opts = random.sample(program_list, 3)
    choices.append({'StudentID': sid, 'Choice1': opts[0], 'Choice2': opts[1], 'Choice3': opts[2]})
choices_df = pd.DataFrame(choices)
choices_df.to_excel(choices_file, index=False, sheet_name='choices')
choices_df.head()

,StudentID,Choice1,Choice2,Choice3
0,UG20250001,UG-EE,UG-LAW,UG-CHEM
1,UG20250002,UG-LAW,UG-ENG,UG-MED
2,UG20250003,UG-CHEM,UG-ARTS,UG-BIO
3,UG20250004,UG-LAW,UG-MATH,UG-CS
4,UG20250005,UG-BIO,UG-BUS,UG-LAW


In [5]:
# Create programme cutoff aggregates (lower is better). We'll assign realistic cutoffs between 6 and 20.
prog_records = []
for prog in program_list:
    cutoff = random.randint(6,20)
    prog_records.append({'ProgrammeCode': prog, 'Cutoff': cutoff, 'ProgrammeName': prog.replace('UG-','').title()})
programmes_df = pd.DataFrame(prog_records)
programmes_df.to_excel(programmes_file, index=False, sheet_name='programmes')
programmes_df

,ProgrammeCode,Cutoff,ProgrammeName
0,UG-CS,8,Cs
1,UG-EE,17,Ee
2,UG-BUS,18,Bus
3,UG-MED,7,Med
4,UG-ENG,8,Eng
5,UG-ARCH,16,Arch
6,UG-LAW,12,Law
7,UG-ARTS,10,Arts
8,UG-SOC,20,Soc
9,UG-BIO,11,Bio


In [6]:
# Load the three sheets
students = pd.read_excel(students_file)
choices = pd.read_excel(choices_file)
programmes = pd.read_excel(programmes_file)

# Merge choices into students
df = students.merge(choices, on='StudentID')

# Helper: get cutoff for a programme code
cutoff_map = programmes.set_index('ProgrammeCode')['Cutoff'].to_dict()
name_map = programmes.set_index('ProgrammeCode')['ProgrammeName'].to_dict()

# Add result columns initialized to 'No'
df['FirstChoice'] = 'No'
df['SecondChoice'] = 'No'
df['ThirdChoice'] = 'No'
df['SuggestedProgramme'] = ''
df['FeePayingSuggestion'] = ''

# Matching logic using if/elif as required
for idx, row in df.iterrows():
    agg = row['Aggregate']
    c1 = row['Choice1']
    c2 = row['Choice2']
    c3 = row['Choice3']
    admitted = None
    # try first choice
    if agg <= cutoff_map.get(c1, 100):
        df.at[idx, 'FirstChoice'] = 'Yes'
        admitted = c1
    elif agg <= cutoff_map.get(c2, 100):
        df.at[idx, 'SecondChoice'] = 'Yes'
        admitted = c2
    elif agg <= cutoff_map.get(c3, 100):
        df.at[idx, 'ThirdChoice'] = 'Yes'
        admitted = c3
    else:
        # not admitted to any of the three choices
        admitted = None

# Suggest programmes for those not admitted to any choice
for idx, row in df.iterrows():
    if df.at[idx, 'FirstChoice'] == 'Yes' or df.at[idx, 'SecondChoice'] == 'Yes' or df.at[idx, 'ThirdChoice'] == 'Yes':
        continue
    agg = row['Aggregate']
    # find programmes where student qualifies (agg <= programme cutoff)
    matches = programmes[programmes['Cutoff'] >= agg].sort_values('Cutoff')
    if not matches.empty:
        # suggest the best-fit programme (lowest cutoff that still >= agg)
        best = matches.iloc[0]['ProgrammeCode']
        df.at[idx, 'SuggestedProgramme'] = best
        # optional fee-paying suggestion: if student's agg is worse than the median cutoff, suggest fee-paying
        median_cutoff = programmes['Cutoff'].median()
        if agg > median_cutoff:
            df.at[idx, 'FeePayingSuggestion'] = 'Consider Fee-Paying Option'
        else:
            df.at[idx, 'FeePayingSuggestion'] = 'Regular Admission'
    else:
        df.at[idx, 'SuggestedProgramme'] = 'None Available'
        df.at[idx, 'FeePayingSuggestion'] = 'Consider Private/Other Institutions'

df.head()

,StudentID,Name,Aggregate,Choice1,Choice2,Choice3,FirstChoice,SecondChoice,ThirdChoice,SuggestedProgramme,FeePayingSuggestion
0,UG20250001,Sika Boateng,26,UG-EE,UG-LAW,UG-CHEM,No,No,No,None Available,Consider Private/Other Institutions
1,UG20250002,Yaa Gyasi,10,UG-LAW,UG-ENG,UG-MED,Yes,No,No,,
2,UG20250003,Esi Mensah,8,UG-CHEM,UG-ARTS,UG-BIO,Yes,No,No,,
3,UG20250004,Esi Annan,3,UG-LAW,UG-MATH,UG-CS,Yes,No,No,,
4,UG20250005,Kofi Dankyi,8,UG-BIO,UG-BUS,UG-LAW,Yes,No,No,,


In [7]:
# Write the results to an excel workbook with multiple sheets
with pd.ExcelWriter(result_file, engine='openpyxl') as writer:
    students.to_excel(writer, sheet_name='original_students', index=False)
    choices.to_excel(writer, sheet_name='original_choices', index=False)
    programmes.to_excel(writer, sheet_name='programmes', index=False)
    df.to_excel(writer, sheet_name='admission_results', index=False)

print('Result written to', result_file)
result_file

Result written to output\admission_results.xlsx


WindowsPath('output/admission_results.xlsx')

**Run instructions:**

1. Install dependencies: `python -m pip install -r requirements.txt`
2. Open and run this notebook in Jupyter. The generated files are stored under `data/` and `output/`.